In [104]:
import pandas as pd
import regex as re
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

dataset = pd.read_csv('Train_dataset.csv')
dataset.head(5)

,Origin,Destination,Flight_Time,Departure Date,Return Date,Duration,Stops,Flight_Name,Price
0,Islamabad,Toronto,4:25am - 2:55pm,2023-05-10,2023-05-10,19h 30m (1 stop),2h 10m in Abu Dhabi (AUH),Etihad Airways,2534
1,Islamabad,Toronto,5:05am - 7:30pm,2023-05-10,2023-05-10,23h 25m (2 stops),3h in Istanbul (IST) - 2h 50m in Frankfurt (FRA),Multiple airlines,3502
2,Islamabad,Toronto,5:05am - 7:30pm,2023-05-10,2023-05-10,23h 25m (2 stops),3h in Istanbul (IST) - 2h 50m in Frankfurt (FRA),Multiple airlines,3662
3,Islamabad,Vancouver,5:05am - 4:45pm,2023-05-10,2023-05-10,23h 40m (2 stops),2h 50m in Istanbul (IST) - 2h in Munich (MUC),Multiple airlines,4883
4,Islamabad,Vancouver,3:35am - 4:45pm,2023-05-10,2023-05-10,25h 10m (2 stops),3h 5m in Doha (DOH) - 2h in Munich (MUC),Multiple airlines,4912


In [105]:
class spliting_cols():
    def __init__(self, dataset):
        self.dataset = dataset
    
    def split_Flight_Time(self):
        dep_per = []
        dep_min = []
        dep_hr = []
        
        land_per = []
        land_min = []
        land_hr = []
        
        for row in self.dataset['Flight_Time']:
            row =  row.split('-')
            dep_time = row[0]
            if "am" in dep_time:
                dep_per.append(0)
                dep_time = dep_time.replace('am', '').split(':')
            else:
                dep_per.append(1)
                dep_time = dep_time.replace('pm', '').split(':')
            
            dep_hr.append(int(dep_time[0]))
            dep_min.append(int(dep_time[1]))
            
            land_time = row[1]
            if "am" in land_time:
                land_per.append(0)
                land_time = land_time.replace('am', '').split(':')
            else:
                land_per.append(1)
                land_time = land_time.replace('pm', '').split(':')
            
            land_hr.append(int(land_time[0]))
            land_min.append(int(land_time[1]))
        
        self.dataset['Dep_per'] = dep_per
        self.dataset['Dep_hr'] = dep_hr
        self.dataset['Dep_min'] = dep_min
        
        self.dataset['Land_per'] = land_per
        self.dataset['Land_hr'] = land_hr
        self.dataset['Land_min'] = land_min
        
#         print(self.dataset.head(5))
    
    def split_date(self):
        dep_Year = []
        dep_Month = []
        dep_Day = []

        ret_Year = []
        ret_Month = []
        ret_Day = []

        cols = ['Departure Date', 'Return Date']
        for row in self.dataset['Departure Date']:
            Date = str(row).replace(' 00:00:00','').split('-')
            dep_Year.append(Date[0])
            dep_Month.append(Date[1])
            dep_Day.append(Date[2])

        for row in self.dataset['Return Date']:
            Date = str(row).replace(' 00:00:00','').split('-')
            ret_Year.append(Date[0])
            ret_Month.append(Date[1])
            ret_Day.append(Date[2])

        self.dataset["Dep_yr"] = dep_Year
        self.dataset["Dep_mn"] = dep_Month
        self.dataset["Dep_dy"] = dep_Day

        self.dataset["Ret_yr"] = ret_Year
        self.dataset["Ret_mn"] = ret_Month
        self.dataset["Ret_dy"] = ret_Day
        
#         cols = ['Dep_yr', 'Dep_mn', 'Dep_dy', 'Ret_yr', 'Ret_mn', 'Ret_dy']
#         print(self.dataset[cols])
        
    def split_duration(self):
        duration_Hours = []
        duration_Min = []
        for row in self.dataset['Duration']:
            temp_text = re.findall(" \([1-9].[a-z\(\)]+", row)
            row = row.replace(temp_text[0], '').split(' ')
            for data in row:
                if "h" in data:
                    duration_Hours.append(int(data.replace('h', '')))
                    duration_Min.append(int(row[1].replace('m', '')))
                    break
                elif "m" in data:
                    duration_Hours.append(0)
                    duration_Min.append(int(data.replace('m', '')))
                    break         
        self.dataset["Dur_hr"] = duration_Hours
        self.dataset["Dur_min"] = duration_Min
        
        cols = ['Dur_hr', 'Dur_min']
#         print(self.dataset[cols])
        
    def split_stops(self):
        total_Stops = []
        for data in self.dataset["Stops"]:
            data = data.split('-')
            total_Stops.append(len(data))

        self.dataset["Stops"] = total_Stops
#         print(self.dataset["Stops"])
    
    ################################################
    # Changing columns datatyes of int64
    ################################################
    
    def change_dtypes(self):
        types = {
                        'Dep_hr':'int64',
                       'Dep_min':'int64',
                       'Dep_per':'int64',
                       'Land_hr':'int64',
                       'Land_min':'int64',
                       'Land_per':'int64',
                        'Dep_yr':'int64',
                       'Dep_mn':'int64',
                        'Dep_dy':'int64',
                        'Ret_yr':'int64',
                       'Ret_mn':'int64',
                        'Ret_dy':'int64',
                       'Dur_hr':'int64',
                        'Dur_min':'int64',
                        'Stops': 'int64',
                         'Price': 'int64',
                       }
        self.dataset = self.dataset.astype(types)
    
    def drop_dumy_cols(self):
        dropCol = ['Flight_Time','Departure Date', 'Return Date', 'Duration']
        self.dataset.drop(dropCol, inplace=True, axis=1)
    
    def get_dataset(self):
        return self.dataset

obj = spliting_cols(dataset)
obj.split_Flight_Time()
obj.split_date()
obj.split_duration()
obj.split_stops()
obj.drop_dumy_cols()
obj.change_dtypes()
dataset = obj.get_dataset()
dataset.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2412 entries, 0 to 2411
Data columns (total 19 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Origin       2412 non-null   object
 1   Destination  2412 non-null   object
 2   Stops        2412 non-null   int64 
 3   Flight_Name  2412 non-null   object
 4   Price        2412 non-null   int64 
 5   Dep_per      2412 non-null   int64 
 6   Dep_hr       2412 non-null   int64 
 7   Dep_min      2412 non-null   int64 
 8   Land_per     2412 non-null   int64 
 9   Land_hr      2412 non-null   int64 
 10  Land_min     2412 non-null   int64 
 11  Dep_yr       2412 non-null   int64 
 12  Dep_mn       2412 non-null   int64 
 13  Dep_dy       2412 non-null   int64 
 14  Ret_yr       2412 non-null   int64 
 15  Ret_mn       2412 non-null   int64 
 16  Ret_dy       2412 non-null   int64 
 17  Dur_hr       2412 non-null   int64 
 18  Dur_min      2412 non-null   int64 
dtypes: int64(16), object(3)
mem

In [106]:
################################################################
# Converting Categorical Data into Numerical Using LabelEncoder
################################################################

# Origin Col to Numeric
label = LabelEncoder()
label.fit(dataset.Origin.drop_duplicates())
dataset.Origin = label.transform(dataset.Origin)

# Destination Col to Numeric
label.fit(dataset.Destination.drop_duplicates())
dataset.Destination = label.transform(dataset.Destination)

# Flight_Name Col to Numeric
label.fit(dataset.Flight_Name.drop_duplicates())
dataset.Flight_Name = label.transform(dataset.Flight_Name)

dataset.head(5)

,Origin,Destination,Stops,Flight_Name,Price,Dep_per,Dep_hr,Dep_min,Land_per,Land_hr,Land_min,Dep_yr,Dep_mn,Dep_dy,Ret_yr,Ret_mn,Ret_dy,Dur_hr,Dur_min
0,0,16,1,5,2534,0,4,25,1,2,55,2023,5,10,2023,5,10,19,30
1,0,16,2,13,3502,0,5,5,1,7,30,2023,5,10,2023,5,10,23,25
2,0,16,2,14,3662,0,5,5,1,7,30,2023,5,10,2023,5,10,23,25
3,0,17,2,14,4883,0,5,5,1,4,45,2023,5,10,2023,5,10,23,40
4,0,17,2,13,4912,0,3,35,1,4,45,2023,5,10,2023,5,10,25,10


In [107]:
#########################################################################
# Splitting Our Dataset into Train and Test Dataset
# Easch Dataset Contain X, and Y for test and Train
# i.e. X_Train, Y_Train, X_Test, Y_Test
#########################################################################

x = dataset.drop('Price', axis=1)
y = dataset[['Price']]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [109]:
#########################################################################
# Applying Linear Regression Model on Dataset and First 5 Predict Values
#########################################################################

Linear_Model = LinearRegression()
Linear_Model.fit(x_train, y_train)

predict = Linear_Model.predict(x_test)
row = [1,2,3,4,5]
predict[row]

array([[ 557.04577527],
       [1998.14995054],
       [ 524.40903283],
       [ 384.54226618],
       [1574.23848247]])

In [110]:
#########################################################################
# Finding the coefficient of Dataset COlumns and Score of Prediction
#########################################################################

score = Linear_Model.score(x_test, predict)
print(f"The Score of Model: {score}\n")

for idx, col_name in enumerate(x_train.columns):
    print(f"the Coefficient of {col_name} is: {Linear_Model.coef_[0][idx]}")

The Score of Model: 1.0

the Coefficient of Origin is: -0.8480862361739396
the Coefficient of Destination is: 31.648934068506936
the Coefficient of Stops is: 465.10363100386854
the Coefficient of Flight_Name is: -9.692420043186525
the Coefficient of Dep_per is: -278.24693134058833
the Coefficient of Dep_hr is: -11.839759971212994
the Coefficient of Dep_min is: 2.6354903805949585
the Coefficient of Land_per is: 194.27711802240478
the Coefficient of Land_hr is: 9.39322441677629
the Coefficient of Land_min is: -0.9725465993281998
the Coefficient of Dep_yr is: 9.947598300641403e-14
the Coefficient of Dep_mn is: 0.0
the Coefficient of Dep_dy is: 303.7376201934517
the Coefficient of Ret_yr is: 0.0
the Coefficient of Ret_mn is: 0.0
the Coefficient of Ret_dy is: -53.227772279217916
the Coefficient of Dur_hr is: 24.82574780767826
the Coefficient of Dur_min is: 2.5979681536998926


In [111]:
dataset = dataset.drop('Price', axis=1)

In [112]:
##############################################################
# Predicting the Price of First 5 Rows in Train Dataset
##############################################################

cols = ['Dur_hr', 'Dur_min']
print(dataset[cols].head(5))
Linear_Model.predict(dataset.head(5))

   Dur_hr  Dur_min
0      19       30
1      23       25
2      23       25
3      23       40
4      25       10


array([[1591.9401133 ],
       [2072.54775391],
       [2062.85533386],
       [2090.705918  ],
       [2174.85502041]])